In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

2024-03-21 12:06:29.138802: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 12:06:29.209427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 12:06:30.298678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
all_players = ['Бот','Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун', 'Миша', 'Ондрей', 'Савва', 'Сеньор', 'Юран']
input_NN_line = [player+"_team1" for player in all_players] + [player+"_team2" for player in all_players]
shown_players =all_players[1:]
# make the list of players from table above
old_method_dataset = pd.read_csv("old_method_dataset.csv")
print (old_method_dataset.head(15))

    Player  Games  Wins  Winrate
0   Сеньор     21    14     0.67
1    Гриша     22    14     0.64
2     Миша     19    11     0.58
3     Ваня     18    10     0.56
4      Бот      6     3     0.50
5     Джун      8     4     0.50
6    Данил     16     8     0.50
7    Вадим     24    10     0.42
8   Ондрей     12     5     0.42
9     Юран     10     4     0.40
10     Ден     18     6     0.33
11   Савва     10     3     0.30


In [3]:
def add_data(names_win, names_lose, campaign='Map', date = ''):
    old_method_dataset = pd.read_csv("old_method_dataset.csv")
    if len(names_win)>len(names_lose):
        names_lose += ['Бот']
    elif len(names_win)<len(names_lose):
        names_win += ['Бот']

    for name in names_win:
        old_method_dataset.loc[old_method_dataset['Player'] == name, 'Wins'] += 1
        old_method_dataset.loc[old_method_dataset['Player'] == name, 'Games'] += 1
    for name in names_lose:
        old_method_dataset.loc[old_method_dataset['Player'] == name, 'Games'] += 1
    old_method_dataset["Winrate"] = old_method_dataset["Wins"] / old_method_dataset["Games"]
    old_method_dataset = old_method_dataset.sort_values(by="Winrate", ascending=False)
    old_method_dataset["Winrate"]=old_method_dataset["Winrate"].apply(lambda x: round(x, 2))

    old_method_dataset.to_csv("old_method_dataset.csv", index=False)
    print(old_method_dataset.to_markdown()) 

    # columns - Date,Map,Player1,Player2,Player3,Player4,Player5,Player6,Player7,Player8,TeamWon
    # add to datasetNN a line with date, map, team1, team2, whichteamwon:
    # e.g. 2024-03-09,Map,Ден,Гриша,Ондрей,Юран,Сеньор,Савва,Данил,Бот,1.0

    while len(names_win)< 4:
        names_win += ['Бот']
    while len(names_lose)< 4:
        names_lose += ['Бот']
    team1 = ','.join(names_win)
    team2 = ','.join(names_lose)
    new_line = f'{date},{campaign},{team1},{team2},1\n'
    with open('datasetNN.csv', 'a') as file:
        file.write(new_line)
    print('Added to datasetNN.csv: ', new_line) 
    return True

In [4]:
def get_teams(present_players):
    if len(present_players)>8: 
        return "Too many players"
    
    all_teams_old_method = pd.DataFrame(columns=["Team1", "Team1 Winrate", "Team2", "Team2 Winrate"])
    all_teams_nn = pd.DataFrame(columns=["Team1", "Team2","NN output"])
    input_nn =pd.DataFrame(columns=input_NN_line)

    if len(present_players)%2==1:
        present_players = present_players + ["Бот"]  

    for i in range(0, len(present_players)):
        for j in range(i+1, len(present_players)):
            for k in range(j+1, len(present_players)):
                if len(present_players)==6:
                    team1 = [present_players[i], present_players[j], present_players[k]]
                    team2 = [x for x in present_players if x not in team1]
                    team1_winrate = old_method_dataset[old_method_dataset["Player"].isin(team1)]["Winrate"].sum()
                    team2_winrate = old_method_dataset[old_method_dataset["Player"].isin(team2)]["Winrate"].sum()
                    all_teams_old_method.loc[len(all_teams_old_method)] = [team1, team1_winrate, team2, team2_winrate]
                    # create a row with the players
                    row = np.zeros(len(input_NN_line))
                    for player in team1:
                        row[input_NN_line.index(player+"_team1")] = 1
                    for player in team2:
                        row[input_NN_line.index(player+"_team2")] = 1
                    # add row to teams_nn
                    input_nn.loc[len(input_nn)] = row
                    all_teams_nn.loc[len(all_teams_nn)] = [team1, team2, 0]
                if len(present_players)==8:
                    for l in range(k+1, len(present_players)):
                        team1 = [present_players[i], present_players[j], present_players[k], present_players[l]]
                        team2 = [x for x in present_players if x not in team1]
                        team1_winrate = old_method_dataset[old_method_dataset["Player"].isin(team1)]["Winrate"].sum()
                        team2_winrate = old_method_dataset[old_method_dataset["Player"].isin(team2)]["Winrate"].sum()
                        all_teams_old_method.loc[len(all_teams_old_method)] = [team1, team1_winrate, team2, team2_winrate]
                        # create a row with the players
                        row = np.zeros(len(input_NN_line))
                        for player in team1:
                            row[input_NN_line.index(player+"_team1")] = 1
                        for player in team2:
                            row[input_NN_line.index(player+"_team2")] = 1
                        # add row to teams_nn
                        input_nn.loc[len(input_nn)] = row
                        all_teams_nn.loc[len(all_teams_nn)] = [team1, team2, 0]
              

    all_teams_old_method.sort_values(by="Team1 Winrate", inplace=True)
    all_teams_old_method.reset_index(drop=True, inplace=True)

    # predict the output of the NN
    model = keras.models.load_model('model.keras')
    all_teams_nn["NN output"] = model.predict(input_nn)
    all_teams_nn.sort_values(by="NN output", inplace=True)

    return all_teams_old_method, all_teams_nn

In [5]:
present_players = ['Вадим', 'Ваня', 'Гриша', 'Ден', 'Савва', 'Сеньор', 'Юран']

In [39]:
results_old, results_nn = get_teams (present_players)
middle_nn = len(results_nn)//2
middle_old = len(results_old)//2

# find closest result to 0.5 in NN
closest = results_nn.iloc[(results_nn["NN output"]-0.5).abs().argsort()[:1]]
#  find the middle entry in the nn
median = results_nn.iloc [[middle_nn]]
median2 = results_nn.iloc [[middle_nn+1]]

# concat previous dataframes
nn_output = pd.concat([closest, median, median2])
# set indeces of nn_output to "Best","Median","Median2"
nn_output.index = ["Best","Median","Median2"]



# find closest result to 0.5 in old method
closest_old = results_old.iloc[[middle_old]]
closest2_old = results_old.iloc[[middle_old+1]]

old_method_output = pd.concat([closest_old, closest2_old])
old_method_output.index = ["Best","Second Best"]



3/3 [==============================] - 0s 5ms/step


In [20]:
import pandas as pd
import numpy as np
from tensorflow import keras
import streamlit as st

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load the model (only executed once!)
@st.cache_resource
def load_model():
	  return keras.models.load_model('model.keras')

all_players = ['Бот','Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун', 'Миша', 'Ондрей', 'Савва', 'Сеньор', 'Юран']
input_NN_line = [player+"_team1" for player in all_players] + [player+"_team2" for player in all_players]

columnsTeam1 = ['Player1','Player2','Player3','Player4']
columnsTeam2 = ['Player5','Player6','Player7','Player8']
columnsPlayers = columnsTeam1 + columnsTeam2

game_threshold = 4 # minimum number of games to be in the dataset as a player
test_size = 0.1 # % of the dataset is used for testing
ModelHasBot = True #  if True, the bot is considered as a player - to consider bot statistics in games 2vs3 or 3vs4
hasMapWeight = False  # if True, the map weight is used in the model
timeStart ='2023-06-18' # start date of the dataset
timeStart=''

map_weights = {
'ливень':5,
'кровавая жатва':5, 
'переход':3,
'мрачный карнавал':5,
'холодный ручей':4,
'похоронный звон':5,
'приход':5,
'смерть в воздухе':5,
'нет милосердию':5,
'вымерший центр':4,
'болотная лихорадка':4,
'последний рубеж':2,
'жертва':3,
'роковой полет':5}

def read_data():
    data=pd.read_csv("datasetNN.csv")
    # preprocess the data
    # Date,Map,Player1,Player2,Player3,Player4,Player5,Player6,Player7,Player8,TeamWon
    data['Date'] = pd.to_datetime(data['Date'])

    if not hasMapWeight:
        data["Map_weight"] =np.ones(len(data)) # remove map weight
    else:
        data["Map_weight"] = data["Map"].apply( lambda x: map_weights[x])
        data["Map_weight"] = data["Map_weight"]/ data["Map_weight"].max()
    #  drop entries which were played before the start date
    if timeStart != '':
        data = data[data['Date'] >= timeStart]
    #  remove game if there is a single player who is not in all_players
        
    data = data[data[columnsPlayers].apply(lambda x: all([player in all_players for player in x]), axis=1)]
    #  reset the index of data
    data.reset_index(drop=True, inplace=True)


    #  create a row with all the players and for each player, if he is in team1 or team2, and add TeamWon
    input_nn = pd.DataFrame(columns=input_NN_line)
    for i in range(0, len(data)):
        row = np.zeros(len(input_NN_line))
        for player in all_players:
            if player in data.iloc[i][columnsTeam1].values:
                row[input_NN_line.index(player+"_team1")] = 1
            if player in data.iloc[i][columnsTeam2].values:
                row[input_NN_line.index(player+"_team2")] = 1
        input_nn.loc[len(input_nn)] = row
    input_nn['TeamWon'] = data['TeamWon']


    #  add data with swapped players
    input_nn_swapped = input_nn.copy()
    for player in all_players:
        input_nn_swapped[player+"_team1"], input_nn_swapped[player+"_team2"] = input_nn_swapped[player+"_team2"], input_nn_swapped[player+"_team1"]
    input_nn_swapped['TeamWon'] = 3 - input_nn_swapped['TeamWon']
    input_nn = pd.concat([input_nn, input_nn_swapped], ignore_index=True)

    return input_nn



def train_model(input_nn):
    # split the data into training and testing
    input_nn = input_nn.sample(frac=1).reset_index(drop=True)
    X = input_nn.drop(columns=["TeamWon"])
    y = input_nn["TeamWon"]-1
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    # create the model
#  create a model
    model = keras.Sequential([
    keras.layers.Dense(100, input_shape=(len(input_NN_line),), activation='relu'),
    keras.layers.Dense(100, activation='relu'), 
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    ])
    from keras.callbacks import EarlyStopping 
    # Early stopping criteria to avoid overtraining
    earlystopping = EarlyStopping(patience=10,
                                verbose=1,
                                restore_best_weights=True)


    # compile the model
    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'],
                # weighted_metrics=['accuracy']
                )
    # train the model
    model.fit(X_train,
            y_train, 
            # sample_weight=w_train,
            validation_data=(X_test, y_test), 
            callbacks=[earlystopping],
            epochs=100)

    return model


def get_teams(present_players):
    if len(present_players)>8: 
        return "Too many players"
    old_method_dataset = pd.read_csv("old_method_dataset.csv")
    all_teams_old_method = pd.DataFrame(columns=["Team1", "Team1 Winrate", "Team2", "Team2 Winrate"])
    all_teams_nn = pd.DataFrame(columns=["Team1", "Team2","NN output"])

    all_players = ['Бот','Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун', 'Миша', 'Ондрей', 'Савва', 'Сеньор', 'Юран']
    input_NN_line = [player+"_team1" for player in all_players] + [player+"_team2" for player in all_players]


    input_nn =pd.DataFrame(columns=input_NN_line)

    if len(present_players)%2==1:
        present_players = present_players + ["Бот"]  

    for i in range(0, len(present_players)):
        for j in range(i+1, len(present_players)):
            for k in range(j+1, len(present_players)):
                if len(present_players)==6:
                    team1 = [present_players[i], present_players[j], present_players[k]]
                    team2 = [x for x in present_players if x not in team1]
                    team1_winrate = old_method_dataset[old_method_dataset["Player"].isin(team1)]["Winrate"].sum()
                    team2_winrate = old_method_dataset[old_method_dataset["Player"].isin(team2)]["Winrate"].sum()
                    all_teams_old_method.loc[len(all_teams_old_method)] = [team1, team1_winrate, team2, team2_winrate]
                    # create a row with the players
                    row = np.zeros(len(input_NN_line))
                    for player in team1:
                        row[input_NN_line.index(player+"_team1")] = 1
                    for player in team2:
                        row[input_NN_line.index(player+"_team2")] = 1
                    # add row to teams_nn
                    input_nn.loc[len(input_nn)] = row
                    all_teams_nn.loc[len(all_teams_nn)] = [team1, team2, 0]
                if len(present_players)==8:
                    for l in range(k+1, len(present_players)):
                        team1 = [present_players[i], present_players[j], present_players[k], present_players[l]]
                        team2 = [x for x in present_players if x not in team1]
                        team1_winrate = old_method_dataset[old_method_dataset["Player"].isin(team1)]["Winrate"].sum()
                        team2_winrate = old_method_dataset[old_method_dataset["Player"].isin(team2)]["Winrate"].sum()
                        all_teams_old_method.loc[len(all_teams_old_method)] = [team1, team1_winrate, team2, team2_winrate]
                        # create a row with the players
                        row = np.zeros(len(input_NN_line))
                        for player in team1:
                            row[input_NN_line.index(player+"_team1")] = 1
                        for player in team2:
                            row[input_NN_line.index(player+"_team2")] = 1
                        # add row to teams_nn
                        input_nn.loc[len(input_nn)] = row
                        all_teams_nn.loc[len(all_teams_nn)] = [team1, team2, 0]
              

    all_teams_old_method.sort_values(by="Team1 Winrate", inplace=True)
    all_teams_old_method.reset_index(drop=True, inplace=True)

    # predict the output of the NN
    model = load_model()
    all_teams_nn["NN output"] = model.predict(input_nn)
    all_teams_nn.sort_values(by="NN output", inplace=True)

    return all_teams_old_method, all_teams_nn


def add_data(names_win, names_lose, campaign='Map', date = ''):
    old_method_dataset = pd.read_csv("old_method_dataset.csv")
    if len(names_win)>len(names_lose):
        names_lose += ['Бот']
    elif len(names_win)<len(names_lose):
        names_win += ['Бот']

    for name in names_win:
        old_method_dataset.loc[old_method_dataset['Player'] == name, 'Wins'] += 1
        old_method_dataset.loc[old_method_dataset['Player'] == name, 'Games'] += 1
    for name in names_lose:
        old_method_dataset.loc[old_method_dataset['Player'] == name, 'Games'] += 1
    old_method_dataset["Winrate"] = old_method_dataset["Wins"] / old_method_dataset["Games"]
    old_method_dataset = old_method_dataset.sort_values(by="Winrate", ascending=False)
    old_method_dataset["Winrate"]=old_method_dataset["Winrate"].apply(lambda x: round(x, 2))

    old_method_dataset.to_csv("old_method_dataset.csv", index=False)

    # columns - Date,Map,Player1,Player2,Player3,Player4,Player5,Player6,Player7,Player8,TeamWon
    # add to datasetNN a line with date, map, team1, team2, whichteamwon:
    # e.g. 2024-03-09,Map,Ден,Гриша,Ондрей,Юран,Сеньор,Савва,Данил,Бот,1.0

    while len(names_win)< 4:
        names_win += ['Бот']
    while len(names_lose)< 4:
        names_lose += ['Бот']
    team1 = ','.join(names_win)
    team2 = ','.join(names_lose)
    new_line = f'{date},{campaign},{team1},{team2},1\n'
    with open('datasetNN.csv', 'a') as file:
        file.write(new_line)
    print('Added to datasetNN.csv: ', new_line) 
    return True

In [21]:

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)

input_nn= read_data()

input_nn.head(25)

,Бот_team1,Вадим_team1,Ваня_team1,Гриша_team1,Данил_team1,Ден_team1,Джун_team1,Миша_team1,Ондрей_team1,Савва_team1,Сеньор_team1,Юран_team1,Бот_team2,Вадим_team2,Ваня_team2,Гриша_team2,Данил_team2,Ден_team2,Джун_team2,Миша_team2,Ондрей_team2,Савва_team2,Сеньор_team2,Юран_team2,TeamWon
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
2,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0
5,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0
6,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
7,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
8,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
9,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
